<a href="https://colab.research.google.com/github/AhmadHegazy/CoronaVirus_Analysis/blob/Development/CoronaVirus_Twitter_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/drive', force_remount=True)

Prepare Twitter Access key from developer account


In [0]:
!ls '/content/drive/My Drive/Colab Notebooks/CoronaVirusAnalysis.config'

'/content/drive/My Drive/Colab Notebooks/CoronaVirusAnalysis.config'


In [0]:
!pip install configParser

In [0]:
import configparser

In [0]:
config = configparser.RawConfigParser()

In [0]:
config.read('/content/drive/My Drive/Colab Notebooks/CoronaVirusAnalysis.config')

['/content/drive/My Drive/Colab Notebooks/CoronaVirusAnalysis.config']

In [0]:
print(config.sections())

['Keys']


In [0]:
accesstoken=config.get('Keys', 'twitter_accessToken')
accesstokensecret=config.get('Keys', 'twitter_accessTokenSecret')
apikey=config.get('Keys', 'twitter_APIKey')
apisecretkey=config.get('Keys', 'twitter_APISecret')

In [0]:
print(accesstoken)
print(accesstokensecret)
print(apikey)
print(apisecretkey)

In [0]:
import tweepy as tw

In [0]:
auth = tw.OAuthHandler(apikey, apisecretkey)
auth.set_access_token(accesstoken, accesstokensecret)
api = tw.API(auth, wait_on_rate_limit=True)

In [0]:
search_word = "#coronavirus"
date_since = "2020-03-10"

In [0]:
tweets = tw.Cursor(api.search,
                   q=search_word,
                   lang='en',
                   since=date_since).items(1500)

In [0]:
for index, tweet in enumerate(tweets):
  print("Index: {} - Tweet: {}".format(index,tweet))
  if index == 10:
    break

In [0]:
tweets_details = [[tweet.geo, tweet.text, tweet.user.screen_name, tweet.user.location] for tweet in tweets]

In [0]:
tweets_details

In [0]:
import pandas as pd

In [0]:
tweet_df = pd.DataFrame(data=tweets_details, columns=['geo','text','user', "location"])

In [0]:
pd.set_option('max_colwidth', 800)
tweet_df.head(20)

In [0]:
tweet_df.location.value_counts()

In [0]:
import re
def clean_tweets(text):
    text = re.sub("RT @[\w]*:","",text)
    text = re.sub("@[\w]*","",text)
    text = re.sub("https?://[A-Za-z0-9./]*","",text)
    text = re.sub("\n","",text)
    return text

In [0]:
tweet_df['text']=tweet_df['text'].apply(lambda x: clean_tweets(x))

In [0]:
tweet_df.head(20)

In [0]:
tweet_df.to_csv('/content/drive/My Drive/Colab Notebooks/coronavirus_tweets.csv')

In [0]:
import spacy 
nlp = spacy.load('en_core_web_sm')

In [0]:
tweet_df['text'].apply(lambda x : [print("\tText: {} , Entity: {}".format(entity.text, entity.label_)) if (not 
entity.text.startswith('#')) else "" for entity in nlp(x).ents])

In [0]:
tweet_df['entities'] = tweet_df['text'].apply(lambda x : [(entity.text, entity.label_) if (not 
entity.text.startswith('#')) else "" for entity in nlp(x).ents])

In [0]:
tweet_df.head(10)

In [0]:
import nltk

In [0]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

In [0]:
sid = SentimentIntensityAnalyzer()

In [0]:
tweet_df['sentiment']=tweet_df['text'].apply(lambda x: sid.polarity_scores(x))

In [0]:
tweet_df.head(10)

In [0]:
!pip install -U googlemaps
import googlemaps

In [0]:
gmaps = googlemaps.Client(key=config.get('Keys','google_API_key'))

In [0]:
geocode_result = gmaps.geocode(tweet_df['location'][1])

In [0]:
print(geocode_result)
print(geocode_result[0]['formatted_address'])

print(geocode_result[0]['formatted_address'].split(",")[-1].strip())

In [0]:
def get_country(input):
  try:
    output=gmaps.geocode(input)[0]['formatted_address'].split(",")[-1].strip()
  except:
    output="Error"
  return output

In [0]:
tweet_df['country'] = tweet_df['location'].apply(lambda x : "" if(not x.strip()) else get_country(x))

In [0]:
tweet_df['country'].value_counts()

In [0]:
tweet_df.count()

In [0]:
tweet_df.isnull().any()

In [0]:
tweet_df.head(10)

In [0]:
tweet_df.to_csv('/content/drive/My Drive/Colab Notebooks/coronavirus_tweets_cleaned.csv')